In [1]:
%load_ext sql
%sql sqlite:///chinook.db
    

'Connected: None@chinook.db'

# Задание 1

Вывести 10 самых больших по размеру треков жанра ROCK и формата MPEG (исоп

In [8]:
%%sql 
SELECT *
FROM tracks
INNER JOIN genres
ON tracks.Genreid = genres.Genreid
INNER JOIN media_types
ON tracks.MediaTypeid = media_types.MediaTypeid
WHERE (genres.name = 'Rock') AND (media_types.name = 'MPEG audio file')
ORDER BY tracks.Bytes DESC
LIMIT 10;

Done.


TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice,GenreId_1,Name_1,MediaTypeId_1,Name_2
1666,Dazed And Confused,137,1,1,Jimmy Page,1612329,52490554,0.99,1,Rock,1,MPEG audio file
620,Space Truckin',50,1,1,Blackmore/Gillan/Glover/Lord/Paice,1196094,39267613,0.99,1,Rock,1,MPEG audio file
1581,Dazed And Confused,127,1,1,Jimmy Page/Led Zeppelin,1116734,36052247,0.99,1,Rock,1,MPEG audio file
2429,We've Got To Get Together/Jingo,198,1,1,None,1070027,34618222,0.99,1,Rock,1,MPEG audio file
2432,Funky Piano,198,1,1,None,934791,30200730,0.99,1,Rock,1,MPEG audio file
621,Going Down / Highway Star,50,1,1,Gillan/Glover/Lord/Nix - Blackmore/Paice,913658,29846063,0.99,1,Rock,1,MPEG audio file
2427,Santana Jam,197,1,1,Carlos Santana,882834,29207100,0.99,1,Rock,1,MPEG audio file
2565,The Sun Road,208,1,1,"Terry Bozzio, Steve Stevens, Tony Levin",880640,29008407,0.99,1,Rock,1,MPEG audio file
1670,Whole Lotta Love,138,1,1,John Bonham/John Paul Jones/Robert Plant/Willie Dixon,863895,28191437,0.99,1,Rock,1,MPEG audio file
622,Mistreated (Alternate Version),50,1,1,Blackmore/Coverdale,854700,27775442,0.99,1,Rock,1,MPEG audio file


## Задание 2
Вывести названия всех групп, их песен и названия их альбомов для всех треков жанра Рок, приобретенные сотрудниками Microsoft.

In [8]:
%%sql
SELECT artists.name as 'Artist', tracks.Name as 'Song', albums.Title as 'Album'
FROM artists
INNER JOIN albums
ON artists.Artistid = albums.Artistid
INNER JOIN tracks
ON albums.Albumid = tracks.Albumid
INNER JOIN genres
ON tracks.Genreid = genres.Genreid
INNER JOIN invoice_items
ON tracks.Trackid = invoice_items.Trackid
INNER JOIN invoices
ON invoice_items.Invoiceid = invoices.Invoiceid
INNER JOIN customers
ON invoices.Customerid = customers.Customerid
WHERE (genres.Name = 'Rock') and (customers.Company = 'Microsoft Corporation')
ORDER BY artists.name;

Done.


Artist,Song,Album
Foo Fighters,No Way Back,In Your Honor [Disc 1]
Foo Fighters,Still,In Your Honor [Disc 2]
Foo Fighters,Razor,In Your Honor [Disc 2]
Foo Fighters,My Hero,The Colour And The Shape
Guns N' Roses,Double Talkin' Jive,Use Your Illusion I
Guns N' Roses,The Garden,Use Your Illusion I
Guns N' Roses,Don't Damn Me,Use Your Illusion I
Guns N' Roses,Dead Horse,Use Your Illusion I


## Задание 3
Для каждого набора (жанр, тип медиа) вывести среднюю цену по стоимости трека и общее количество, причем вывести только те наборы, для которых все треку стоят больше 1,5$.


In [6]:
%%sql
SELECT genres.Name as 'Genre', media_types.Name as 'MediaType', AVG(tracks.UnitPrice) as 'Average price', count(tracks.TrackId) as 'Number of tracks'
FROM genres, media_types
LEFT JOIN tracks
ON genres.genreid = tracks.genreid and media_types.MediaTypeId = tracks.MediaTypeId
GROUP BY genres.Name, media_types.Name
having MIN(tracks.UnitPrice) > 1.5


Done.


Genre,MediaType,Average price,Number of tracks
Comedy,Protected MPEG-4 video file,1.9899999999999995,17
Drama,Protected MPEG-4 video file,1.9899999999999978,64
Sci Fi & Fantasy,Protected MPEG-4 video file,1.9900000000000004,26
Science Fiction,Protected MPEG-4 video file,1.9899999999999995,13
TV Shows,Protected MPEG-4 video file,1.990000000000001,93


## Задание 4
Вывести компании, сделавшие максимальное и минимальное число заказов.

In [9]:
%%sql
WITH Name AS(
SELECT Company, COUNT(Company) AS Quentity 
FROM customers
JOIN Invoices 
ON customers.CustomerId = invoices.CustomerId
WHERE Company <> 'None'
GROUP BY Company)
    
SELECT *
FROM Name
WHERE Quentity = (SELECT MIN(Quentity) FROM Name)
UNION
SELECT *
FROM Name
WHERE  Quentity = (SELECT MAX(Quentity) FROM Name) 
    

Done.


Company,Quentity
Apple Inc.,7
Banco do Brasil S.A.,7
Embraer - Empresa Brasileira de Aeronáutica S.A.,7
Google Inc.,7
JetBrains s.r.o.,7
Microsoft Corporation,7
Riotur,7
Rogers Canada,7
Telus,7
Woodstock Discos,7


## Задание 5
Для каждой компании вывести общее количестов песен, купленных по жанру поп-музыки

In [7]:
%%sql
SELECT customers.Company as 'Company', COUNT(*) as 'Number of songs'
FROM customers
INNER JOIN invoices
ON customers.CustomerId = invoices.CustomerId
INNER JOIN invoice_items
ON invoices.InvoiceId = invoice_items.InvoiceId
INNER JOIN tracks
ON invoice_items.TrackId = tracks.TrackId
INNER JOIN genres
ON tracks.GenreId = genres.GenreId
WHERE (genres.Name = 'Pop') and (customers.Company <> 'None')
GROUP BY customers.Company;



Done.


Company,Number of songs
Embraer - Empresa Brasileira de Aeronáutica S.A.,2
JetBrains s.r.o.,4


## Задание 6
Вывести средний размер альбома в байтах.

In [11]:
%%sql
WITH Albums2 AS (SELECT distinct albums.Title as 'Album', SUM(tracks.Bytes) as "Sum"
FROM albums
INNER JOIN tracks
ON albums.AlbumId = tracks.AlbumId
GROUP BY albums.title)

SELECT AVG(Sum) as 'Average size'
FROM Albums2


Done.


Average size
338288920.3170029


In [12]:
%%sql
SELECT AVG(Sum) as 'Average size'
FROM(
SELECT distinct albums.Title as 'Album', SUM(tracks.Bytes) as "Sum"
FROM albums
INNER JOIN tracks
ON albums.AlbumId = tracks.AlbumId
GROUP BY albums.title)




Done.


Average size
338288920.3170029
